In [1]:
from pathlib import Path
from langchain_community.document_loaders import TextLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
import torch
import pickle

In [2]:
import json

In [3]:
torch.cuda.is_available()
torch.set_float32_matmul_precision('high')

In [7]:
splitting_model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embedding_model_name =  "intfloat/multilingual-e5-base"

In [8]:
splitting_embeddings = HuggingFaceEmbeddings(
    model_name=splitting_model_name,
    model_kwargs={
        'device': 'cuda',
    },
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
text_splitter = SemanticChunker(
    embeddings=splitting_embeddings,
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=0.9,
    )

In [6]:
docs_dir = Path("/home/bartek/Kod/PD/praca_dyplomowa/dane/texts/cleaned_texts")
documents = []

In [7]:
for txt_file in list(docs_dir.glob("*"))[:10]:
    print(txt_file)

/home/bartek/Kod/PD/praca_dyplomowa/dane/texts/cleaned_texts/Taiwan_Government_Targeted_by_Multiple_Cyberattacks_in_April_2020_Part_2:_Owlproxy_Malware
/home/bartek/Kod/PD/praca_dyplomowa/dane/texts/cleaned_texts/Bzz.._Bzz.
/home/bartek/Kod/PD/praca_dyplomowa/dane/texts/cleaned_texts/Prying_Libra
/home/bartek/Kod/PD/praca_dyplomowa/dane/texts/cleaned_texts/Beware_Android_trojan_posing_as_Clubhouse_app
/home/bartek/Kod/PD/praca_dyplomowa/dane/texts/cleaned_texts/Reflections_of_the_Israel-Palestine_Conflict_on_the_Cyber_World
/home/bartek/Kod/PD/praca_dyplomowa/dane/texts/cleaned_texts/RAT_Gone_Rogue:_Meet_ARS_VBS_Loader
/home/bartek/Kod/PD/praca_dyplomowa/dane/texts/cleaned_texts/TeaBot_Banking_Trojan_Posted_as_QR_Code_app_in_Google_Play_Store_Targeting_US_Users
/home/bartek/Kod/PD/praca_dyplomowa/dane/texts/cleaned_texts/Joker
/home/bartek/Kod/PD/praca_dyplomowa/dane/texts/cleaned_texts/Qakbot_Botnet_Disruption
/home/bartek/Kod/PD/praca_dyplomowa/dane/texts/cleaned_texts/Mustang_Panda’

In [16]:
for dir in docs_dir.glob("*"):
    txt_file = Path(dir).glob("LLM_clean_text*.txt")
    loader = TextLoader(list(txt_file)[0], encoding="utf-8")
    loaded_docs = loader.load()

    json_file = Path(f"{dir}/meta_data.json")
    with json_file.open("r", encoding="utf-8") as f:
        meta_from_json = json.load(f)
    
    for doc in loaded_docs:
        doc.metadata["title"] = meta_from_json["title"]
        doc.metadata["language"] = meta_from_json["language"]
        doc.metadata["url"] = meta_from_json["url"]
        doc.metadata["urldate"] = meta_from_json["urldate"]
        

        
    documents.extend(loaded_docs)

print(f"Loaded {len(documents)} documents")

Loaded 8911 documents


In [17]:
split_docs = text_splitter.split_documents(documents)

In [ ]:
splits_file = Path("/home/bartek/Kod/PD/praca_dyplomowa/dane/vectordb/split_documents.pkl")
with open(splits_file, 'wb') as f:
    pickle.dump(split_docs, f)

In [19]:
del splitting_embeddings
del text_splitter
torch.cuda.empty_cache()

In [20]:
with open(splits_file, 'rb') as f:
    split_docs = pickle.load(f)

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-base",
    model_kwargs={
        'device': 'cuda',
        'trust_remote_code': True
    },
    encode_kwargs={'normalize_embeddings': True,
                   "batch_size": 8}
)

In [22]:
vectorstore = Chroma.from_documents(
    documents=split_docs,
    embedding=embeddings,
    collection_name="malpedia_vectors",
    persist_directory="./chroma_db"
)